In [5]:

# !pip install asyncio
# !pip install nest-asyncio
# !pip install pandas
# !pip install tqdm

In [1]:
# API_KEY = "zgJAdEg3lkVxGXyEtM89tOZplOsTi8bv" # Personal
API_KEY = "pqmKVrIjJjkKQMhvRslPapP7QzNV2A1I"# Hackathon
notebook_dir = "C:\\Users\\William\\Documents\\ML\\a16z Hackathon\\A16ZxMistral\\single_image_test"
image_foler = "images"
images_dir = f"{notebook_dir}/{image_foler}"

In [2]:

import os
from mistralai import Mistral
import base64
import asyncio
import nest_asyncio
nest_asyncio.apply()
import time
from tqdm.notebook import tqdm

model = "pixtral-12b-latest"
client = Mistral(api_key=API_KEY)

In [3]:
def encode_image_base64(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode("utf-8")



In [4]:
files = os.listdir(images_dir)
print(files)
encoded_images = [encode_image_base64(f"{images_dir}/{file}") for file in files]
prompts = [
	{
		"name": "fire",
		"prompt": "Do you see fire in this image? If so return 'True' otherwise return 'False' without the quotation marks."
	},
	{
		"name": "smoke",
		"prompt": "Do you see smoke or dense steam in the iamge ? If so return 'True' otherwise return 'False' without the quotation marks."
	},
	{
		"name": "police",
		"prompt": "Do you see police or law enforcement present in the image? If so return 'True' otherwise return 'False' without the quotation marks."
	},
	{
		"name": "Formations",
		"prompt": "Do you see people orginised in formations e.g. standing in organised lines? If so return 'True' otherwise return 'False' without the quotation marks."
	},
	{
		"name": "crowds",
		"prompt": "Do you see any crowds of people of 20 or more? If so return 'True' otherwise return 'False' without the quotation marks."
	},
	{
		"name": "crush",
		"prompt": "Do you see any crowds of more than 60 people, that could be dangerous to vunerable people? If so return 'True' otherwise return 'False' without the quotation marks."
	},
]



['Crowd.png', 'Fire.png', 'Line of Police.png', 'Line of Protesters.png', 'No Crowd.png', 'No FIre.png', 'No tags.png', 'small images.zip', 'Smoke.png']


In [5]:

async def get_pixtral_reponse(encoded_image, prompt):
    print(f"Prompt: {prompt['name']}")
    try_number = 0
    while True: 
        try:
            start = time.time()
            chat_response = await asyncio.to_thread(
                client.chat.complete,  # Run blocking call in a separate thread
                model=model,
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": f"{prompt['prompt']}"
                            },
                            {
                                "type": "image_url",
                                "image_url": f"data:image/png;base64,{encoded_image}"
                            },
                        ]
                    },
                ]
            )
            response = chat_response.choices[0].message.content
            end = time.time()
            print(f"Attempt: {try_number} Time {end - start:.2f}s")
            break
        except Exception as e: 
            print(f"Error: {e}, retrying... Attempt: {try_number}")
            try_number += 1
            await asyncio.sleep(1)  # Introduce a short delay before retrying
    return response

async def analyse_image(encoded_image, prompts):
    coroutines = [
        get_pixtral_reponse(encoded_image, prompt) 
        for prompt in tqdm(prompts)
    ]
    results = await asyncio.gather(*coroutines)
    return results

async def analyse_images(encoded_images, prompts):
    coroutines = [
        analyse_image(encoded_image, prompts) 
        for encoded_image in tqdm(encoded_images)
    ]
    results = await asyncio.gather(*coroutines)
    return results

In [6]:
single_result = await get_pixtral_reponse(encoded_images[0], prompts[0])
print(single_result )

Prompt: fire
Error: [SSL: SSLV3_ALERT_BAD_RECORD_MAC] sslv3 alert bad record mac (_ssl.c:2578), retrying... Attempt: 0
Attempt: 1 Time 1.87s
False


In [7]:
multiple_prompts = asyncio.run(analyse_image(encoded_images[0], prompts))
print(multiple_prompts)

  0%|          | 0/6 [00:00<?, ?it/s]

Prompt: fire
Prompt: smoke
Prompt: police
Prompt: Formations
Prompt: crowds
Prompt: crush
Error: Server disconnected without sending a response., retrying... Attempt: 0
Error: Server disconnected without sending a response., retrying... Attempt: 0
Error: [SSL: SSLV3_ALERT_BAD_RECORD_MAC] sslv3 alert bad record mac (_ssl.c:2578), retrying... Attempt: 0
Error: Server disconnected without sending a response., retrying... Attempt: 0
Error: Server disconnected without sending a response., retrying... Attempt: 0
Error: Server disconnected without sending a response., retrying... Attempt: 1
Error: Server disconnected without sending a response., retrying... Attempt: 1
Error: Server disconnected without sending a response., retrying... Attempt: 1
Error: Server disconnected without sending a response., retrying... Attempt: 1
Attempt: 0 Time 2.24s
Attempt: 1 Time 1.59s
Error: Server disconnected without sending a response., retrying... Attempt: 2
Error: Server disconnected without sending a resp

In [8]:
final_results = asyncio.run(analyse_images(encoded_images, prompts))

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

Prompt: fire
Prompt: smoke
Prompt: police
Prompt: Formations
Prompt: crowds
Prompt: crush
Prompt: fire
Prompt: smoke
Prompt: police
Prompt: Formations
Prompt: crowds
Prompt: crush
Prompt: fire
Prompt: smoke
Prompt: police
Prompt: Formations
Prompt: crowds
Prompt: crush
Prompt: fire
Prompt: smoke
Prompt: police
Prompt: Formations
Prompt: crowds
Prompt: crush
Prompt: fire
Prompt: smoke
Prompt: police
Prompt: Formations
Prompt: crowds
Prompt: crush
Prompt: fire
Prompt: smoke
Prompt: police
Prompt: Formations
Prompt: crowds
Prompt: crush
Prompt: fire
Prompt: smoke
Prompt: police
Prompt: Formations
Prompt: crowds
Prompt: crush
Prompt: fire
Prompt: smoke
Prompt: police
Prompt: Formations
Prompt: crowds
Prompt: crush
Prompt: fire
Prompt: smoke
Prompt: police
Prompt: Formations
Prompt: crowds
Prompt: crush
Attempt: 0 Time 2.14s
Attempt: 0 Time 3.00s
Attempt: 0 Time 2.71s
Attempt: 0 Time 2.83s
Attempt: 0 Time 3.08s
Attempt: 0 Time 3.52s
Attempt: 0 Time 4.37s
Attempt: 0 Time 4.37s
Attempt: 0 Tim

KeyboardInterrupt: 

In [ ]:
# final_results
columns = [prompt["name"] for prompt in prompts]
rows = [file.split(".")[0] for file in files]

import pandas as pd

df = pd.DataFrame(final_results, columns=columns, index=rows)
print(df)


NameError: name 'prompts' is not defined